## Example for only AF8

### Import packages & load data

In [109]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from scipy.signal import iirnotch, butter, filtfilt
from scipy.stats import skew, ttest_ind
import mne
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../data/5min_256Hz_17:11:23.csv')

In [110]:
# first glance
data.head()

,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,Gyro_X,Gyro_Y,Gyro_Z,HeadBandOn,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements
0,2023-11-17 09:37:56.372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/muse/event/connected Muse-AB4C
1,2023-11-17 09:37:56.451,0.316438,0.948555,0.766601,1.154738,-0.153094,0.388197,0.431739,0.71507,0.398981,...,6.25061,-3.132782,-0.687866,1.0,1.0,1.0,1.0,1.0,45.0,NaN
2,2023-11-17 09:37:56.452,0.316438,0.948555,0.766601,1.154738,-0.153094,0.388197,0.431739,0.71507,0.398981,...,6.25061,-3.132782,-0.687866,1.0,1.0,1.0,1.0,1.0,45.0,NaN
3,2023-11-17 09:37:56.452,0.316438,0.948555,0.766601,1.154738,-0.153094,0.388197,0.431739,0.71507,0.398981,...,6.25061,-3.132782,-0.687866,1.0,1.0,1.0,1.0,1.0,45.0,NaN
4,2023-11-17 09:37:56.453,0.316438,0.948555,0.766601,1.154738,-0.153094,0.388197,0.431739,0.71507,0.398981,...,6.25061,-3.132782,-0.687866,1.0,1.0,1.0,1.0,1.0,45.0,NaN


In [111]:
df.shape

(87326, 40)

In [112]:
# TODO: handle missing values in a better fashion (interpolation?)
channels = ['RAW_AF8']
af8 = data[channels]
af8 = af8.dropna()

### Artifact removal

In [113]:
# Removing the 50Hs power line artifact
fs = 256
f0 = 50
quality_factor = 30

b, a = iirnotch(f0, quality_factor, fs)
af8['AF8_filt'] = filtfilt(b, a, af8['RAW_AF8'])

### Normalize and clamp
Method from: https://www.nature.com/articles/s42256-023-00714-5


(extended in: https://static-content.springer.com/esm/art%3A10.1038%2Fs42256-023-00714-5/MediaObjects/42256_2023_714_MOESM1_ESM.pdf)

In [114]:
mean = af8['AF8_filt'].mean()
std = af8['AF8_filt'].std()

# Clamp values greater than 20 standard deviations
af8['AF8_filt_normal'] = np.clip(af8['AF8_filt'], mean - 20*std, mean + 20*std)

# Apply a robust scaler
scaler = RobustScaler(quantile_range=(25.0, 75.0))
af8['AF8_filt_normal'] = scaler.fit_transform(af8[['AF8_filt_normal']])

### Band pass filter

In [115]:
# Parameters
lowcut = 0.5
highcut = 50.0
fs = 256  
filter_order = 5

# Butterworth filter
nyquist_freq = 0.5 * fs
low = lowcut / nyquist_freq
high = highcut / nyquist_freq
b, a = butter(filter_order, [low, high], btype="band")
af8['AF8_bandpassed'] = filtfilt(b, a, af8['AF8_filt_normal'])

In [116]:
af8.head()

,RAW_AF8,AF8_filt,AF8_filt_normal,AF8_bandpassed
1,824.798535,824.903110,4.572145,0.118288
2,819.157509,819.329414,3.859365,-0.402088
3,818.351648,818.350359,3.734161,-0.650035
4,820.769231,820.599158,4.021744,-0.598607
5,821.575092,821.480203,4.134414,-0.441593


In [117]:
# Custom function to calculate approximate entropy
def approximate_entropy(segment, m=2, r=None):
    if not r:
        r = 0.2 * np.std(segment)
    
    def phi(m):
        x = np.array([segment[j:j+m] for j in range(len(segment) - m + 1)])
        C = np.sum([np.sum(np.abs(x - x[i]) <= r, axis=1) / (len(segment) - m + 1) for i in range(len(segment) - m + 1)], axis=0) / (len(segment) - m + 1)
        return np.sum(np.log(C)) / (len(segment) - m + 1)
    
    return np.abs(phi(m) - phi(m + 1))

In [16]:
# Define the segment size
segment_size = int(0.8 * 256) # this is 204 which is 0.796875 seconds

# Create empty lists to store our results
feature_data = {}

for i in range(0, len(data), segment_size):
    for channel in channels:
        segment = data[channel].iloc[i:i+segment_size]
        if len(segment) < segment_size: continue
        segment = segment.reset_index(drop=True)
        feature_data[f'{channel}_approx_entropy'] = feature_data.get(f'{channel}_approx_entropy', []) + [approximate_entropy(segment)]
        feature_data[f'{channel}_total_variation'] = feature_data.get(f'{channel}_total_variation', []) + [np.sum(np.gradient(segment))]
        feature_data[f'{channel}_standard_variation'] = feature_data.get(f'{channel}_standard_variation', []) + [np.std(segment)]
        feature_data[f'{channel}_energy'] = feature_data.get(f'{channel}_energy', []) + [np.sum(segment**2)]
        feature_data[f'{channel}_skewness'] = feature_data.get(f'{channel}_skewness', []) + [skew(segment)]


# You can now convert these lists into a DataFrame
features_df = pd.DataFrame(feature_data)

/var/folders/vn/vg9g17yx0hgcwsgfx9z89kpc0000gn/T/ipykernel_96375/3492679977.py:16: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  feature_data[f'{channel}_skewness'] = feature_data.get(f'{channel}_skewness', []) + [skew(segment)]


In [17]:
print(features_df.shape)
print(features_df.head())

(427, 25)
   Delta_AF8_approx_entropy  Delta_AF8_total_variation  \
0                  0.411689                  -1.084486   
1                  0.414500                   0.428411   
2                  0.411689                   0.551657   
3                  0.420183                  -0.034374   
4                  0.409269                  -0.500206   

   Delta_AF8_standard_variation  Delta_AF8_energy  Delta_AF8_skewness  \
0                      0.399570         44.587422           -0.148714   
1                      0.134545          4.476283           -0.381333   
2                      0.201296         41.978996           -0.169881   
3                      0.013328         87.341659           -0.311431   
4                      0.195606         36.077862           -0.010685   

   Theta_AF8_approx_entropy  Theta_AF8_total_variation  \
0                  0.415488                  -0.511263   
1                  0.414662                   0.191133   
2                  0.409406 

### Add previous windows (context for time series)

In [23]:
def add_windows(feature_df, num_windows=9):
    new_features = []

    for i in range(len(feature_df)):
        current_window = feature_df.iloc[i]
        if i < num_windows:
            previous_windows = pd.DataFrame(index=range(num_windows - i), columns=feature_df.columns)
            previous_windows = previous_windows.fillna(0)
            previous_windows = pd.concat([previous_windows, feature_df.iloc[:i]])
        else:
            previous_windows = feature_df.iloc[i - num_windows:i]
        concatenated_features = pd.concat([current_window, previous_windows.unstack()]).reset_index(drop=True)
        new_features.append(concatenated_features)
    
    return pd.DataFrame(new_features)

new_df = add_windows(features_df)

In [19]:
print(new_df.shape)
print(new_df.head())

(427, 250)
        0         1         2          3         4         5         6    \
0  0.411689 -1.084486  0.399570  44.587422 -0.148714  0.415488 -0.511263   
1  0.414500  0.428411  0.134545   4.476283 -0.381333  0.414662  0.191133   
2  0.411689  0.551657  0.201296  41.978996 -0.169881  0.409406  0.368375   
3  0.420183 -0.034374  0.013328  87.341659 -0.311431  0.420768 -0.108639   
4  0.409269 -0.500206  0.195606  36.077862 -0.010685  0.409269 -0.678909   

        7          8         9    ...        240  241  242  243  244  245  \
0  0.188802   9.243903  0.486909  ...   0.000000  0.0  0.0  0.0  0.0  0.0   
1  0.067259   0.945109  0.563417  ...  37.162192  0.0  0.0  0.0  0.0  0.0   
2  0.140974  25.118489 -0.248611  ...  32.159219  0.0  0.0  0.0  0.0  0.0   
3  0.033168  40.093959 -0.482902  ...  15.662252  0.0  0.0  0.0  0.0  0.0   
4  0.262520  14.102580  0.047215  ...  25.543372  0.0  0.0  0.0  0.0  0.0   

        246       247       248       249  
0  0.000000  0.000000  0.

### Join with labels

In [1]:
# calculate error tolerance

# Given parameters
eeg_window_size = 0.796875  # seconds
session_duration = 10 * 60  # 10 minutes in seconds
error_margin = 0.01  # seconds, assumed acceptable error margin

# Calculate the number of image transitions in a perfectly timed session
perfect_transitions = session_duration / eeg_window_size

# Calculate the number of image transitions with the error margin
min_transitions_with_error = session_duration / (eeg_window_size + error_margin)
max_transitions_with_error = session_duration / (eeg_window_size - error_margin)

# Calculate the difference in the number of transitions
difference_min = perfect_transitions - min_transitions_with_error
difference_max = max_transitions_with_error - perfect_transitions

perfect_transitions, min_transitions_with_error, max_transitions_with_error, difference_min, difference_max



(752.9411764705883,
 743.6096049573973,
 762.5099285146943,
 9.331571513190966,
 9.568752044105963)

### Pick relevant feautures with the t-test

In [ ]:
# TODO: join new_df with labels

# Split data
in_the_zone = data[data['label'] == 1]
out_of_the_zone = data[data['label'] == 0]

# Perform t-tests
features = data.columns.drop('label')
p_values = {}
for feature in features:
    stat, p_val = ttest_ind(in_the_zone[feature], out_of_the_zone[feature])
    p_values[feature] = p_val

# Select significant features
significant_features = [feature for feature, p_val in p_values.items() if p_val < 0.05]

# Output significant features
print(significant_features)